# Lesson 3: Enable Logging

### Import all needed packages

In [1]:
import boto3
import json
import os

bedrock = boto3.client('bedrock', region_name="us-west-2")

In [2]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [3]:
log_group_name = '/my/amazon/bedrock/logs'

In [4]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' created successfully.


In [5]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [6]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': '9b15b1f5-f829-4ec3-bdc1-7d86b21b21b6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 19 Mar 2024 12:44:27 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9b15b1f5-f829-4ec3-bdc1-7d86b21b21b6'},
  'RetryAttempts': 0}}

In [7]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': '5ef2b229-7251-40b6-b8fd-a466a9b4c858',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 19 Mar 2024 12:44:58 GMT',
   'content-type': 'application/json',
   'content-length': '572',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5ef2b229-7251-40b6-b8fd-a466a9b4c858'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::355985022847:role/c99355a2566044l6195406t1w35598502284-LoggingIAMRole-bMH2lPQt4Bc0',
   'largeDataDeliveryS3Config': {'bucketName': 'c99355a2566044l6195406t1w355985022-loggings3bucket-znumykrkcvhd',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'c99355a2566044l6195406t1w355985022-loggings3bucket-znumykrkcvhd',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [8]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [9]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet that is the setting of many science fiction stories and movies. It is often described as a planet that is similar to Earth in terms of size and composition, but with some unique features that make it a fascinating and dangerous place to explore. One of the most notable features of Foobar is its atmosphere. The planet's atmosphere is thick and toxic, making it difficult for humans to breathe without specialized equipment. The atmosphere is also home to a wide range of exotic creatures, including giant insects, floating jellyfish, and mysterious monsters. Despite the dangers of its atmosphere, Foobar is also home to a thriving human civilization. The planet's inhabitants have developed advanced technology and culture, and they have built impressive cities and structures that soar high into the sky. One of the most famous cities on Foobar is Metropolis, a sprawling metropolis that is home to millions of people and is ruled by a powerful emperor. Metropolis is

In [11]:
cloudwatch.print_recent_logs(log_group_name)

Permissions are correctly set for Amazon Bedrock logs.
-------------------------

{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-03-19T12:45:50Z",
    "accountId": "355985022847",
    "identity": {
        "arn": "arn:aws:sts::355985022847:assumed-role/voclabs/user3166701=eyJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcHAiLCJzdWIiOiIyNDk"
    },
    "region": "us-west-2",
    "requestId": "2b017a41-3949-407e-a603-4a0dc32a01e1",
    "operation": "InvokeModel",
    "modelId": "amazon.titan-text-express-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "Write an article about the fictional planet Foobar.",
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        },
        "inputTokenCount": 10
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJ

To review the logs within the AWS console, please use the following link to reference the steps outlined in the video:

In [12]:
from IPython.display import HTML
aws_url = os.environ['AWS_CONSOLE_URL']

In [ ]:
HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')
